In [ ]:
# Dependencies and Setup
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json


In [ ]:
# Check if data has been pickled and if so load pickle
# else pickle only the attributes we are interested in, and compress
# pickle is much faster/smaller than csv
if os.path.exists('data/txDF.pkl') :
    txDF=pd.read_pickle('data/txDF.pkl',compression='gzip')
else:
    df = pd.read_csv('data/US_Accidents_Dec19.csv',encoding='utf-8')
    columns_of_interest=[
        'ID','Severity','Start_Time','End_Time','Start_Lat', 'Start_Lng', 'City', 
        'County', 'State','Zipcode', 'Country','Visibility(mi)', 
        'Weather_Condition','Precipitation(in)','Sunrise_Sunset','Civil_Twilight'
    ]
    condition=df['State'] =='TX'
    txDF=df[condition][columns_of_interest] 
    txDF.to_pickle("data/txDF.pkl",compression='gzip')

In [ ]:
txDF.columns

In [ ]:
txDF['City'].value_counts()

In [ ]:
txDF